## Importing the required libraries

In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from tensorflow.keras import optimizers
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

In [2]:
from keras import regularizers

In [3]:
import os

In [4]:
folders = os.listdir('C:/emotions')

In [5]:
type(folders)

list

In [6]:
print(folders)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24', 'RawData']


In [7]:
print(folders[0:24])


['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [8]:
rawFolder = folders[24]
rawAudioFiles = os.listdir('C:/emotions/' + rawFolder)

In [9]:
print(len(rawAudioFiles))

1920


## Plotting the audio file's waveform and its spectrogram

In [10]:
import os
import pandas as pd
import librosa
import glob 

In [11]:
import matplotlib.pyplot as plt
import scipy.io.wavfile
import numpy as np
import sys

## Setting the labels

In [12]:
feeling_list=[]
for item in rawAudioFiles:
    if item[6:-16]=='02' and int(item[18:-4])%2==0:
        feeling_list.append('female_calm')
    elif item[6:-16]=='02' and int(item[18:-4])%2==1:
        feeling_list.append('male_calm')
    elif item[6:-16]=='03' and int(item[18:-4])%2==0:
        feeling_list.append('female_happy')
    elif item[6:-16]=='03' and int(item[18:-4])%2==1:
        feeling_list.append('male_happy')
    elif item[6:-16]=='04' and int(item[18:-4])%2==0:
        feeling_list.append('female_sad')
    elif item[6:-16]=='04' and int(item[18:-4])%2==1:
        feeling_list.append('male_sad')
    elif item[6:-16]=='05' and int(item[18:-4])%2==0:
        feeling_list.append('female_angry')
    elif item[6:-16]=='05' and int(item[18:-4])%2==1:
        feeling_list.append('male_angry')
    elif item[6:-16]=='06' and int(item[18:-4])%2==0:
        feeling_list.append('female_fearful')
    elif item[6:-16]=='06' and int(item[18:-4])%2==1:
        feeling_list.append('male_fearful')
    elif item[:1]=='a':
        feeling_list.append('male_angry')
    elif item[:1]=='f':
        feeling_list.append('male_fearful')
    elif item[:1]=='h':
        feeling_list.append('male_happy')
    #elif item[:1]=='n':
        #feeling_list.append('neutral')
    elif item[:2]=='sa':
        feeling_list.append('male_sad')

In [13]:
labels = pd.DataFrame(feeling_list)

In [14]:
labels[:10]

,0
0,male_calm
1,female_calm
2,male_calm
3,female_calm
4,male_calm
5,female_calm
6,male_calm
7,female_calm
8,male_calm
9,female_calm


In [43]:
len(rawAudioFiles)

1920

## Getting the features of audio files using librosa

In [16]:
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(rawAudioFiles):
    if rawAudioFiles[index][6:-16]!='01' and rawAudioFiles[index][6:-16]!='07' and rawAudioFiles[index][6:-16]!='08' and rawAudioFiles[index][:2]!='su' and rawAudioFiles[index][:1]!='n' and rawAudioFiles[index][:1]!='d':
        X, sample_rate = librosa.load('C:/emotions/RawData/'+ y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        #[float(i) for i in feature]
        #feature1=feature[:135]
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1        

In [42]:
len(df)

1200

In [18]:
df3 = pd.DataFrame(df['feature'].values.tolist())

In [19]:
df3[:5]

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,...,-58.802044,-57.447464,-58.896500,-58.750996,-57.405678,-60.078484,-63.426800,-62.638542,-61.082737,-60.234661
1,-65.707649,-65.707649,-63.114719,-61.518997,-61.097141,-63.424599,-63.720066,-56.854614,-55.168972,-54.639999,...,-38.301201,-39.792141,-40.613159,-41.209202,-41.439201,-43.994278,-49.399620,-50.591599,-49.144051,-48.705654
2,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,...,-29.193260,-31.346556,-34.310772,-35.800705,-35.936115,-37.631844,-40.119411,-41.662888,-41.323643,-40.710770
3,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-65.928223,...,-48.872002,-48.674301,-48.596073,-47.602745,-43.049198,-42.659542,-43.188560,-44.001244,-43.610100,-44.698246
4,-62.364311,-59.934727,-61.869602,-67.495773,-71.071808,-65.679817,-63.394402,-65.503349,-61.856644,-60.005428,...,-33.817356,-39.071327,-41.897121,-40.865437,-38.290604,-36.372398,-37.915779,-40.026127,-43.383774,-43.965401


In [20]:
newdf = pd.concat([df3,labels], axis=1)

In [21]:
rnewdf = newdf.rename(index=str, columns={"0": "label"})

In [22]:
rnewdf

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
0,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,-70.267769,...,-57.447464,-58.896500,-58.750996,-57.405678,-60.078484,-63.426800,-62.638542,-61.082737,-60.234661,male_calm
1,-65.707649,-65.707649,-63.114719,-61.518997,-61.097141,-63.424599,-63.720066,-56.854614,-55.168972,-54.639999,...,-39.792141,-40.613159,-41.209202,-41.439201,-43.994278,-49.399620,-50.591599,-49.144051,-48.705654,female_calm
2,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,-65.482498,...,-31.346556,-34.310772,-35.800705,-35.936115,-37.631844,-40.119411,-41.662888,-41.323643,-40.710770,male_calm
3,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-64.528450,-65.928223,...,-48.674301,-48.596073,-47.602745,-43.049198,-42.659542,-43.188560,-44.001244,-43.610100,-44.698246,female_calm
4,-62.364311,-59.934727,-61.869602,-67.495773,-71.071808,-65.679817,-63.394402,-65.503349,-61.856644,-60.005428,...,-39.071327,-41.897121,-40.865437,-38.290604,-36.372398,-37.915779,-40.026127,-43.383774,-43.965401,male_calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,-24.313856,-24.041178,-25.228422,-26.094988,-24.132362,-25.118191,-25.258566,-24.432186,-24.738892,-24.688087,...,-8.436266,-10.197830,-12.263414,-14.144474,-7.916656,-3.283535,-2.470568,-2.487463,-1.785601,male_sad
1196,-27.733097,-25.317772,-24.269955,-25.217028,-24.375731,-23.234041,-23.060852,-23.593578,-25.465048,-24.390287,...,-19.613743,-22.958599,-22.451134,-22.969593,-24.166462,-23.357229,-23.805897,-22.046549,-21.264595,male_sad
1197,-26.477873,-21.849876,-11.200557,-6.713976,-5.940035,-6.923090,-7.600639,-7.370038,-7.653201,-9.667166,...,-7.755277,-6.800155,-6.931998,-8.070164,-6.287150,-5.732646,-5.968968,-2.897266,-0.117553,male_sad
1198,-28.242472,-26.161549,-24.954851,-23.735605,-26.121582,-25.552244,-22.299961,-8.970101,-3.758892,-3.458179,...,-25.844707,-25.865776,-22.196074,-22.867290,-23.568983,-23.386150,-24.064566,-22.987600,-23.478584,male_sad


In [23]:
from sklearn.utils import shuffle
rnewdf = shuffle(newdf)
rnewdf[:10]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
161,-66.471031,-66.464798,-66.471031,-66.471031,-66.471031,-66.471031,-66.471031,-66.471031,-66.471031,-66.471031,...,-64.386414,-62.351788,-64.222397,-66.152023,-63.642780,-63.769035,-66.471031,-66.471031,-65.012711,female_calm
101,-70.152740,-69.913689,-67.810699,-68.664963,-71.470451,-71.470451,-71.470451,-71.470451,-71.470451,-71.470451,...,-43.779472,-43.768883,-44.085487,-43.975651,-44.347027,-44.098366,-46.298855,-47.547016,-46.484295,female_calm
1053,-21.392101,-21.662266,-22.259338,-24.444981,-23.050682,-23.140684,-22.903954,-23.729919,-24.871449,-24.049696,...,-21.701115,-11.972590,-9.322908,-10.145143,-12.772147,-14.352438,-16.328117,-12.141912,-6.584519,male_fearful
636,-58.458244,-58.458244,-56.495502,-54.816059,-55.097450,-56.864162,-58.341858,-58.458244,-58.458244,-57.983540,...,-56.273376,-55.921200,-55.327827,-56.125870,-54.906818,-55.580368,-55.196865,-55.768448,-57.050266,male_angry
729,-43.144321,-43.240807,-43.574074,-43.559708,-43.377747,-42.510563,-42.398952,-41.682373,-41.117344,-41.384140,...,-32.683876,-33.158852,-33.733822,-35.125347,-37.479668,-36.947750,-36.216888,-34.943890,-37.373890,female_angry
1034,-19.608782,-17.110071,-16.288795,-11.379373,-4.384048,-2.022365,-2.588405,-4.300551,-4.970526,-6.814387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male_fearful
1057,-25.793974,-25.637810,-25.623514,-24.908133,-24.204029,-23.441559,-22.504555,-23.215464,-23.309681,-22.387897,...,-8.886435,-10.918305,-11.311975,-10.998633,-9.800959,-9.060637,-9.556483,-3.739459,3.027152,male_fearful
344,-46.874268,-47.988571,-48.316895,-48.316895,-48.316895,-48.316895,-48.316895,-48.316895,-48.316895,-48.316895,...,-44.549980,-43.942745,-43.958191,-44.920956,-45.609703,-45.971531,-44.832127,-44.833458,-46.158363,male_happy
155,-66.002304,-66.021881,-66.983719,-64.977615,-62.615215,-62.309586,-62.451180,-60.948093,-59.981323,-59.848747,...,-46.775917,-48.598087,-48.730759,-47.965942,-49.415318,-50.584885,-53.095871,-55.438869,-54.290329,female_calm
123,-62.435238,-60.987801,-60.403915,-60.022331,-59.200264,-59.942162,-59.997593,-58.442886,-58.532730,-60.239304,...,-52.922703,-51.896187,-54.785355,-56.142796,-53.869633,-54.963013,-55.977486,-57.662533,-56.375557,female_calm


In [24]:
rnewdf=rnewdf.fillna(0)

## Dividing the data into test and train

In [25]:
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]

In [26]:
test

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0
729,-43.144321,-43.240807,-43.574074,-43.559708,-43.377747,-42.510563,-42.398952,-41.682373,-41.117344,-41.384140,...,-32.683876,-33.158852,-33.733822,-35.125347,-37.479668,-36.947750,-36.216888,-34.943890,-37.373890,female_angry
1057,-25.793974,-25.637810,-25.623514,-24.908133,-24.204029,-23.441559,-22.504555,-23.215464,-23.309681,-22.387897,...,-8.886435,-10.918305,-11.311975,-10.998633,-9.800959,-9.060637,-9.556483,-3.739459,3.027152,male_fearful
898,-42.110771,-44.450138,-43.878517,-43.572796,-44.328049,-47.081234,-44.381809,-43.837570,-44.099277,-43.878906,...,-43.238144,-45.474014,-45.770714,-47.273411,-47.923321,-44.996952,-44.425571,-43.073017,-41.846512,male_fearful
230,-53.980492,-52.021362,-52.914917,-53.042610,-52.869320,-55.149212,-60.087891,-56.723209,-54.732597,-56.222458,...,-51.273087,-51.264603,-49.336857,-49.406067,-49.028599,-47.317200,-48.304489,-46.692009,-41.980919,male_happy
983,-27.551817,-25.248302,-22.912870,-23.200481,-24.561083,-24.916075,-24.402773,-23.608076,-24.458662,-26.309509,...,-24.333981,-24.204628,-24.982929,-24.918051,-26.273071,-26.806255,-27.242727,-26.979715,-27.205202,male_angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,-58.327255,-54.953262,-55.248722,-53.967308,-51.944901,-52.322750,-53.839474,-54.893929,-51.611965,-52.376587,...,-43.847286,-43.556648,-42.028385,-40.557636,-43.097683,-42.761173,-42.750996,-45.633850,-44.306820,male_calm
856,-45.593971,-45.288883,-43.909744,-43.639015,-44.616287,-43.998116,-43.588974,-44.078384,-43.900879,-42.735794,...,-40.460434,-40.520290,-40.441990,-41.478268,-42.777439,-41.823460,-41.062496,-40.644089,-42.000237,male_fearful
487,-52.527290,-52.740135,-51.810162,-50.845955,-52.180595,-50.952663,-51.928787,-50.068733,-51.063869,-51.178474,...,-52.454109,-53.226479,-51.588139,-48.858006,-49.559284,-49.241310,-49.745724,-50.692467,-49.275898,female_sad
256,-53.141991,-52.674992,-50.214226,-50.819004,-54.603825,-52.255264,-50.683800,-52.724796,-49.598228,-51.657570,...,-47.976353,-47.613281,-48.414055,-49.194748,-50.277706,-52.465252,-54.078613,-56.517986,-57.023308,male_happy


In [27]:
trainfeatures = train.iloc[:, :-1]

In [28]:
trainlabel = train.iloc[:, -1:]

In [29]:
testfeatures = test.iloc[:, :-1]

In [30]:
testlabel = test.iloc[:, -1:]

In [31]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)

lb = LabelEncoder()



In [32]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(944, 216)
(944, 1)
(256, 216)
(256, 1)


In [33]:
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))


C:\Users\Simas\Miniconda3\envs\med4\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [34]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(944, 216)
(944, 10)
(256, 216)
(256, 10)


In [35]:
X_train.shape

(944, 216)

## Padding sequence for CNN model

In [36]:
print('Pad sequences')
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)

Pad sequences


In [37]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 216, 128)          768       
_________________________________________________________________
activation (Activation)      (None, 216, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 216, 128)          82048     
_________________________________________________________________
activation_1 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout (Dropout)            (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 27, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 27, 128)           8

In [39]:
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [41]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=32, epochs=1009, validation_data=(x_testcnn, y_test))

Epoch 1/1009
30/30 [==============================] - 3s 72ms/step - loss: 2.5833 - accuracy: 0.0660 - val_loss: 2.3190 - val_accuracy: 0.0352
Epoch 2/1009
30/30 [==============================] - 2s 59ms/step - loss: 2.3503 - accuracy: 0.0930 - val_loss: 2.2990 - val_accuracy: 0.0977
Epoch 3/1009
30/30 [==============================] - 2s 59ms/step - loss: 2.3261 - accuracy: 0.1183 - val_loss: 2.2870 - val_accuracy: 0.1562
Epoch 4/1009
30/30 [==============================] - 2s 59ms/step - loss: 2.3096 - accuracy: 0.1308 - val_loss: 2.2814 - val_accuracy: 0.1719
Epoch 5/1009
30/30 [==============================] - 2s 59ms/step - loss: 2.2674 - accuracy: 0.1674 - val_loss: 2.2692 - val_accuracy: 0.1758
Epoch 6/1009
30/30 [==============================] - 2s 62ms/step - loss: 2.2860 - accuracy: 0.1360 - val_loss: 2.2620 - val_accuracy: 0.1953
Epoch 7/1009
30/30 [==============================] - 2s 59ms/step - loss: 2.2682 - accuracy: 0.1410 - val_loss: 2.2581 - val_accuracy: 0.1875

KeyboardInterrupt: 

## Plotting the accuracy and loss graph

In [ ]:
#sigmoid
plt.plot(cnnhistory.history['accuracy'])
plt.plot(cnnhistory.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Saving the model

In [ ]:
model_name = 'Emotion_Voice_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

## Predicting emotions on the test data

In [ ]:
preds = model.predict(x_testcnn, 
                         batch_size=32, 
                         verbose=1)

In [ ]:
preds

In [ ]:
preds1=preds.argmax(axis=1)

In [ ]:
preds1

In [ ]:
abc = preds1.astype(int).flatten()

In [ ]:
predictions = (lb.inverse_transform((abc)))

In [ ]:
preddf = pd.DataFrame({'predictedvalues': predictions})
preddf[:10]

In [ ]:
actual=y_test.argmax(axis=1)
abc123 = actual.astype(int).flatten()
actualvalues = (lb.inverse_transform((abc123)))

In [ ]:
actualdf = pd.DataFrame({'actualvalues': actualvalues})
actualdf[:10]

In [ ]:
finaldf = actualdf.join(preddf)

## Actual v/s Predicted emotions

In [ ]:
finaldf[170:180]

In [ ]:
finaldf.groupby('actualvalues').count()

In [ ]:
finaldf.groupby('predictedvalues').count()

In [ ]:
finaldf.to_csv('Predictions.csv', index=False)